## RANDOM FOREST

In [1]:
# ============================================
# RANDOM FOREST — PRECISION-CONTROLLED v5s4
# (SSD/HDD) — Big-mode en TRAIN y TEST (streaming)
#   • FIX: NaT-safe en BIG loader
#   • TEST 2024 streaming (sin cargar todo a RAM)
#   • HDD big-mode: sin rolling (delta, cummax, age_days)
# ============================================

import os, gc, json, warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import pyarrow.parquet as pq

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve, average_precision_score, precision_score,
    recall_score, f1_score, confusion_matrix
)
import joblib

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None


# ===================== UTILS =====================

def cleanup():
    gc.collect()

def downcast_df(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if pd.api.types.is_float_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast="float")
        elif pd.api.types.is_integer_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], downcast="integer")
    return df


# ===================== IO Helpers =====================

def _list_parquet_files(path: str) -> List[str]:
    if os.path.isdir(path):
        return [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".parquet")]
    return [path]

def _peek_row_group(path: str) -> pd.DataFrame:
    # Lee un row-group chiquito para inspeccionar columnas
    files = _list_parquet_files(path)
    for f in files:
        pf = pq.ParquetFile(f)
        if pf.num_row_groups > 0:
            tb = pf.read_row_group(0, columns=None)
            return tb.to_pandas().head(100)
    return pd.DataFrame()

def discover_smart_columns(path: str) -> List[str]:
    peek = _peek_row_group(path)
    if peek.empty:
        return []
    cols = peek.columns.tolist()
    return sorted([c for c in cols if ("smart" in c.lower()) and c.endswith("_raw")])


def _iter_parquet_row_groups(path: str, years: List[int], columns: Optional[List[str]] = None):
    files = _list_parquet_files(path)
    for f in files:
        pf = pq.ParquetFile(f)
        for i in range(pf.num_row_groups):
            tb = pf.read_row_group(i, columns=columns) if columns else pf.read_row_group(i)
            df = tb.to_pandas()
            if "date" not in df.columns:  # ignora row-groups sin fecha
                continue
            df["date"] = pd.to_datetime(df["date"], errors="coerce")
            mask = df["date"].dt.year.isin(years)
            if mask.any():
                yield downcast_df(df.loc[mask].reset_index(drop=True))
            del df, tb
            cleanup()


def load_data(path: str, years: List[int], columns: Optional[List[str]] = None) -> pd.DataFrame:
    print(f"Loading {years} from {path}...")
    chunks = []
    for df in _iter_parquet_row_groups(path, years, columns=columns):
        chunks.append(df)
        if len(chunks) >= 16:
            chunks = [pd.concat(chunks, ignore_index=True)]
            cleanup()
    out = pd.concat(chunks, ignore_index=True) if chunks else pd.DataFrame()
    print(f"Loaded {len(out):,} rows")
    return out


# ===================== BIG MODE (TRAIN y TEST) =====================

def scan_fail_dates(path: str, years: List[int]) -> Dict[str, pd.Timestamp]:
    """Primera pasada: PRIMERA fecha de fallo por serial."""
    print("Scanning earliest failure dates (streaming)...")
    fail_map: Dict[str, pd.Timestamp] = {}
    for df in _iter_parquet_row_groups(path, years, columns=["serial_number", "date", "failure"]):
        df = df[df["failure"] == 1]
        if df.empty:
            continue
        sns = df["serial_number"].astype(str).values
        dts = pd.to_datetime(df["date"], errors="coerce")
        for sn, dt in zip(sns, dts):
            if pd.isna(dt):
                continue
            if sn not in fail_map or dt < fail_map[sn]:
                fail_map[sn] = dt
    print(f"  Found {len(fail_map):,} failed serials")
    return fail_map


def load_data_big_filtered(
    path: str,
    years: List[int],
    fail_map: Dict[str, pd.Timestamp],
    lookahead_days: int,
    hard_window: int,
    neg_random_keep_rate: float = 0.0025,
    columns: Optional[List[str]] = None,
) -> pd.DataFrame:
    """
    TRAIN big: conserva todos positivos, negativos cercanos y una muestra de lejanos.
    (Evita traer 200M+ filas).
    """
    print(f"Loading BIG-FILTERED {years} from {path} (keep_rate={neg_random_keep_rate})...")
    rng = np.random.default_rng(42)
    kept = []
    total_rows = 0
    kept_rows = 0

    for df in _iter_parquet_row_groups(path, years, columns=columns):
        total_rows += len(df)
        sn_ser = df["serial_number"].astype(str)
        dt = pd.to_datetime(df["date"], errors="coerce").to_numpy(dtype="datetime64[D]")

        # map fail date y dtf (NaT-safe)
        fail_series = sn_ser.map(fail_map)
        fdt = pd.to_datetime(fail_series, errors="coerce").to_numpy(dtype="datetime64[D]")

        dtf = np.full(len(df), 10**9, dtype=np.int64)
        valid = ~np.isnat(fdt)
        if valid.any():
            dd = (fdt[valid] - dt[valid]).astype("timedelta64[D]").astype("int64")
            dtf[valid] = dd

        failure = (df["failure"].values == 1)
        pos_mask = failure | ((dtf >= 0) & (dtf <= lookahead_days))
        near_mask = (dtf > lookahead_days) & (dtf <= hard_window)

        keep = pos_mask | near_mask
        far_neg = (~keep) & (~failure)
        if far_neg.any():
            # muestreo aleatorio de negativos lejanos
            sample = rng.random(far_neg.sum()) < neg_random_keep_rate
            sel = np.zeros_like(far_neg, dtype=bool)
            sel[np.where(far_neg)[0]] = sample
            keep = keep | sel

        kept.append(df.loc[keep].reset_index(drop=True))
        kept_rows += int(keep.sum())

        if sum(len(x) for x in kept) > 3_000_000:
            kept = [pd.concat(kept, ignore_index=True)]
            cleanup()

    out = pd.concat(kept, ignore_index=True) if kept else pd.DataFrame()
    rate = 100 * kept_rows / max(1, total_rows)
    print(f"  BIG-FILTERED kept {kept_rows:,}/{total_rows:,} rows (~{rate:.2f}%)")
    return out


# ===================== PREP =====================

def prepare_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["serial_number"] = df["serial_number"].astype(str)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["serial_number", "date"])
    df = df.sort_values(["serial_number", "date"]).reset_index(drop=True)
    return df


# ===================== LABELS =====================

def compute_days_to_failure(dfs: pd.DataFrame) -> np.ndarray:
    fail_map: Dict[str, pd.Timestamp] = {}
    fails = dfs[dfs["failure"] == 1]
    for sn, dt in zip(fails["serial_number"], fails["date"]):
        fail_map[sn] = min(dt, fail_map.get(sn, dt))

    dtf = np.full(len(dfs), 10**9, dtype=np.int64)
    for i, (sn, dt) in enumerate(zip(dfs["serial_number"], dfs["date"])):
        if sn in fail_map:
            dtf[i] = (fail_map[sn] - dt).days
    return dtf


def create_labels_from_dtf(dtf: np.ndarray, lookahead: int = 7) -> np.ndarray:
    return ((dtf >= 0) & (dtf <= lookahead)).astype(np.int8)


# ===================== CATEGÓRICOS =====================

def extract_vendor(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()
    return s.str.extract(r"^([A-Za-z]+)", expand=False).fillna("UNK")

def fit_category_maps(df: pd.DataFrame) -> Dict[str, Dict[str, int]]:
    maps: Dict[str, Dict[str, int]] = {}
    if "model" in df.columns:
        models = pd.Index(df["model"].astype(str).unique())
        maps["model"] = {m: i for i, m in enumerate(models)}
        vendors = pd.Index(extract_vendor(df["model"]).unique())
        maps["vendor"] = {v: i for i, v in enumerate(vendors)}
    return maps

def apply_category_maps(df: pd.DataFrame, maps: Optional[Dict[str, Dict[str, int]]]) -> Tuple[pd.Series, pd.Series]:
    if maps and "model" in df.columns and "model" in maps and "vendor" in maps:
        m = df["model"].astype(str)
        v = extract_vendor(m)
        model_map = maps["model"]
        vendor_map = maps["vendor"]
        model_codes = m.map(model_map).fillna(-1).astype(int)
        vendor_codes = v.map(vendor_map).fillna(-1).astype(int)
        return model_codes, vendor_codes
    n = len(df)
    return pd.Series(np.zeros(n, dtype=np.int32), index=df.index), pd.Series(np.zeros(n, dtype=np.int32), index=df.index)


# ===================== FEATURES =====================

def create_features_joined(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    dataset_type: str,
    add_rolling: bool,
    fit_cats_on_train: bool = True,
    category_maps: Optional[Dict[str, Dict[str, int]]] = None,
) -> Tuple[pd.DataFrame, pd.DataFrame, List[str], Dict[str, Dict[str, int]]]:
    """Modo estándar (RAM): TRAIN∪TEST → features → split."""
    print(f"Creating features (temporal join) for {dataset_type}...")

    df_train = df_train.copy(); df_train["__subset__"] = "train"
    df_test  = df_test.copy();  df_test["__subset__"]  = "test"
    df_all = pd.concat([df_train, df_test], ignore_index=True)
    df_all.sort_values(["serial_number", "date"], inplace=True)

    all_cols = df_all.columns.tolist()
    smart_cols = [c for c in all_cols if ("smart" in c.lower()) and ("_raw" in c.lower())]
    print(f"  Found {len(smart_cols)} SMART attributes")

    for c in smart_cols:
        df_all[c] = pd.to_numeric(df_all[c], errors="coerce").fillna(0)

    for c in smart_cols:
        df_all[f"delta_{c}"] = df_all.groupby("serial_number", sort=False)[c].diff().fillna(0)
        df_all[f"max_{c}"]   = df_all.groupby("serial_number", sort=False)[c].cummax()

    if add_rolling:
        for c in smart_cols:
            r = df_all.groupby("serial_number", sort=False)[c]
            df_all[f"rmean7_{c}"] = r.rolling(window=7, min_periods=2).mean().reset_index(level=0, drop=True).fillna(0)
            df_all[f"rstd7_{c}"]  = r.rolling(window=7, min_periods=2).std().reset_index(level=0, drop=True).fillna(0)

    df_all["age_days"] = df_all.groupby("serial_number", sort=False).cumcount()
    d = df_all["date"]
    df_all["month"] = d.dt.month
    df_all["day_of_week"] = d.dt.dayofweek

    if fit_cats_on_train:
        cat_maps = fit_category_maps(df_train)
    else:
        cat_maps = category_maps or {}
    model_code, vendor_code = apply_category_maps(df_all, cat_maps)
    df_all["model_code"] = model_code
    df_all["vendor_code"] = vendor_code

    drop_cols = ["serial_number", "date", "failure", "model"]
    X_all = df_all.drop(columns=[c for c in drop_cols if c in df_all.columns], errors="ignore")

    for c in X_all.columns:
        if X_all[c].dtype == "object":
            X_all[c] = pd.Categorical(X_all[c]).codes
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").fillna(0)

    var = X_all.var()
    keep = var[var > 0].index.tolist()
    X_all = X_all[keep].astype(np.float32)

    X_train = X_all[df_all["__subset__"] == "train"].reset_index(drop=True)
    X_test  = X_all[df_all["__subset__"] == "test"].reset_index(drop=True)

    print(f"  Final features: {X_all.shape[1]}")
    return X_train, X_test, keep, cat_maps


# ========== STREAMING FEATURES (TEST BIG MODE, sin rolling) ==========

class StreamDeltaCummaxBuilder:
    """
    Construye features de TEST en streaming (sin rolling).
    Mantiene por disco:
      • last_vals[smart]   • cummax_vals[smart]   • age_days
    """
    def __init__(self, smart_cols: List[str], cat_maps: Dict[str, Dict[str, int]]):
        self.smart_cols = smart_cols
        self.cat_maps = cat_maps
        self.last_vals: Dict[str, Dict[str, float]] = {}     # serial -> {smart: last}
        self.cummax_vals: Dict[str, Dict[str, float]] = {}   # serial -> {smart: cummax}
        self.age: Dict[str, int] = {}                        # serial -> age counter

    def _ensure_serial(self, sn: str):
        if sn not in self.last_vals:
            self.last_vals[sn] = {c: 0.0 for c in self.smart_cols}
            self.cummax_vals[sn] = {c: 0.0 for c in self.smart_cols}
            self.age[sn] = 0

    def transform_chunk(self, df: pd.DataFrame) -> pd.DataFrame:
        # Orden por serial, date
        df = df.sort_values(["serial_number", "date"]).reset_index(drop=True)

        # Categóricos
        model_code, vendor_code = apply_category_maps(df, self.cat_maps)
        df["model_code"] = model_code
        df["vendor_code"] = vendor_code

        # Calendario
        df["month"] = df["date"].dt.month
        df["day_of_week"] = df["date"].dt.dayofweek

        # Inicializa features
        for c in self.smart_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

        # Calcula delta/cummax/age en streaming por serial
        deltas = {f"delta_{c}": [] for c in self.smart_cols}
        cummaxs = {f"max_{c}": [] for c in self.smart_cols}
        ages = []

        serials = df["serial_number"].astype(str).values
        for idx, sn in enumerate(serials):
            self._ensure_serial(sn)
            ages.append(self.age[sn])
            self.age[sn] += 1

            for c in self.smart_cols:
                v = float(df.at[idx, c])
                d = v - self.last_vals[sn][c]
                self.last_vals[sn][c] = v
                self.cummax_vals[sn][c] = max(self.cummax_vals[sn][c], v)
                deltas[f"delta_{c}"].append(d)
                cummaxs[f"max_{c}"].append(self.cummax_vals[sn][c])

        for k, arr in deltas.items():
            df[k] = arr
        for k, arr in cummaxs.items():
            df[k] = arr
        df["age_days"] = ages

        # Deja solo columnas de features posibles
        feat_cols = (
            self.smart_cols
            + [f"delta_{c}" for c in self.smart_cols]
            + [f"max_{c}" for c in self.smart_cols]
            + ["age_days", "month", "day_of_week", "model_code", "vendor_code"]
        )
        return df[feat_cols].astype(np.float32)


# ===================== CV, MODEL, MÉTRICAS =====================

def make_group_folds(serials: pd.Series, y: np.ndarray, n_splits: int = 5, random_state: int = 42):
    serials = serials.astype(str).values
    uniq_serials, inverse = np.unique(serials, return_inverse=True)
    y_disk = np.zeros(len(uniq_serials), dtype=np.int8)
    for idx_row, disk_idx in enumerate(inverse):
        if y[idx_row] == 1:
            y_disk[disk_idx] = 1
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for tr_d, va_d in skf.split(uniq_serials, y_disk):
        tr_mask = np.isin(inverse, tr_d)
        va_mask = np.isin(inverse, va_d)
        yield np.where(tr_mask)[0], np.where(va_mask)[0]


def sample_negatives_hard(
    X: pd.DataFrame, y: np.ndarray, dtf: np.ndarray, lookahead: int,
    neg_pos_ratio: int = 3, hard_window: int = 60, hard_fraction: float = 0.7,
    seed: int = 42,
):
    rng = np.random.default_rng(seed)
    pos_idx = np.where(y == 1)[0]
    if len(pos_idx) == 0:
        raise ValueError("No positives in training fold for hard-negative sampling")

    n_pos = len(pos_idx)
    n_neg_needed = n_pos * neg_pos_ratio

    hard_mask = (dtf > lookahead) & (dtf <= hard_window)
    hard_idx = np.where(hard_mask & (y == 0))[0]
    easy_idx = np.where(~hard_mask & (y == 0))[0]

    n_hard = min(int(n_neg_needed * hard_fraction), len(hard_idx))
    n_easy = min(n_neg_needed - n_hard, len(easy_idx))

    chosen_hard = rng.choice(hard_idx, size=n_hard, replace=False) if n_hard > 0 else np.empty(0, dtype=int)
    chosen_easy = rng.choice(easy_idx, size=n_easy, replace=False) if n_easy > 0 else np.empty(0, dtype=int)

    sel_idx = np.concatenate([pos_idx, chosen_hard, chosen_easy])
    Xb = X.iloc[sel_idx].reset_index(drop=True)
    yb = y[sel_idx]
    return Xb, yb


def train_rf(
    X: pd.DataFrame, y: np.ndarray,
    n_estimators: int = 500, max_depth: int = 16,
    min_samples_leaf: int = 20, max_features: float | str = 0.3,
    random_state: int = 42
) -> RandomForestClassifier:
    print(f"  Train RF: n={len(y):,} (pos={int(y.sum()):,}) | trees={n_estimators} depth={max_depth} leaf={min_samples_leaf}")
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight=None,
        n_jobs=-1,
        random_state=random_state,
        verbose=0,
    )
    rf.fit(X, y)
    return rf


def pick_threshold(
    y_true: np.ndarray, proba: np.ndarray,
    min_precision: float = 0.90, min_recall: float = 0.10,
    top_k_rate: float = 1e-4, min_alerts: int = 5
):
    precision, recall, thr = precision_recall_curve(y_true, proba)
    pr_auc = average_precision_score(y_true, proba)

    valid = (precision >= min_precision) & (recall >= min_recall)
    if valid.any():
        idx = int(np.argmax(recall * valid))
        chosen = thr[idx if idx < len(thr) else len(thr)-1]
    else:
        beta = 0.5
        fbeta = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall + 1e-10)
        idx = int(np.argmax(fbeta))
        chosen = thr[idx if idx < len(thr) else len(thr)-1]

    alerts = int((proba >= chosen).sum())
    if alerts < max(1, min_alerts):
        if top_k_rate and top_k_rate > 0:
            k = max(max(1, min_alerts), int(len(proba) * top_k_rate))
            chosen = float(np.partition(proba, -k)[-k])
        else:
            chosen = float(np.quantile(proba, 0.9999))

    return float(chosen), float(pr_auc)


def metrics_at_threshold(y_true: np.ndarray, proba: np.ndarray, thr: float) -> Dict:
    y_pred = (proba >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return {
        'precision': float(precision_score(y_true, y_pred, zero_division=0)),
        'recall': float(recall_score(y_true, y_pred, zero_division=0)),
        'f1': float(f1_score(y_true, y_pred, zero_division=0)),
        'confusion_matrix': [[int(tn), int(fp)], [int(fn), int(tp)]],
        'fpr': float(fp / (fp + tn)) if (fp + tn) > 0 else 0.0
    }


# ===================== MAIN PIPELINE =====================

def train_random_forest_precision_pipeline(
    train_parquet: str,
    test_parquet: str | None,
    dataset_type: str,
    train_years: List[int] = [2020, 2021, 2022, 2023],
    test_years: List[int] | None = [2024],
    lookahead_days: int = 7,
    n_splits: int = 5,
    neg_pos_ratio: int = 3,
    hard_window: int = 60,
    hard_fraction: float = 0.7,
    rf_n_estimators: int = 500,
    rf_max_depth: int = 16,
    rf_min_samples_leaf: int = 20,
    rf_max_features: float | str = 0.3,
    min_precision: float = 0.85,
    min_recall: float = 0.05,
    top_k_rate: float = 2e-4,
    min_alerts: int = 20,
    output_dir: str = './models_rf_precision_v5s4',
    random_state: int = 42,
    # BIG mode switches
    big_mode: bool = False,                 # TRAIN big
    test_big_mode: bool = False,            # TEST big
    neg_random_keep_rate: float = 0.0025,   # TRAIN big keep rate
    add_rolling: bool = True,               # usa rolling? (desactivar en HDD big)
):
    print("="*100)
    print(f"RANDOM FOREST (PRECISION-CONTROLLED v5s4) - {dataset_type.upper()}")
    print("="*100)
    os.makedirs(output_dir, exist_ok=True)

    # Descubrir columnas SMART para leer solo lo necesario
    smart_cols = discover_smart_columns(train_parquet)
    base_cols = ['serial_number', 'date', 'failure', 'model']
    read_cols = base_cols + smart_cols

    # ---------- Load TRAIN ----------
    if big_mode:
        fail_map_train = scan_fail_dates(train_parquet, train_years)
        df_train_raw = load_data_big_filtered(
            train_parquet, train_years, fail_map_train,
            lookahead_days=lookahead_days, hard_window=hard_window,
            neg_random_keep_rate=neg_random_keep_rate, columns=read_cols
        )
    else:
        df_train_raw = load_data(train_parquet, train_years, columns=read_cols)

    # ---------- Load TEST (si no es big, RAM) ----------
    if test_parquet and test_years and (not test_big_mode):
        df_test_raw = load_data(test_parquet, test_years, columns=read_cols)
    else:
        df_test_raw = pd.DataFrame()  # TEST big se hace en streaming más adelante

    if df_train_raw.empty:
        raise ValueError("Training data is empty!")

    # ---------- Prepare ----------
    df_train = prepare_df(df_train_raw)
    df_test  = prepare_df(df_test_raw) if not df_test_raw.empty else pd.DataFrame()

    # ---------- Labels / DTF ----------
    dtf_all = compute_days_to_failure(df_train)
    y_all = create_labels_from_dtf(dtf_all, lookahead_days)
    print(f"Labels (train): {int(y_all.sum()):,} positive ({100*y_all.mean():.4f}%)")
    if y_all.sum() < 50:
        raise ValueError(f"Insufficient positive samples in TRAIN: {y_all.sum()}")

    # ---------- Features (TRAIN, y TEST si no es big) ----------
    X_all, X_test, feature_names, cat_maps = create_features_joined(
        df_train, df_test, dataset_type,
        add_rolling=add_rolling,
        fit_cats_on_train=True, category_maps=None
    )

    # ---------- Grouped CV en TRAIN ----------
    serials_all = df_train['serial_number']
    oof_proba = np.zeros(len(y_all), dtype=np.float32)
    fold_metrics = []

    for fold, (tr_idx, va_idx) in enumerate(make_group_folds(serials_all, y_all, n_splits=n_splits, random_state=random_state), start=1):
        X_tr, y_tr = X_all.iloc[tr_idx].reset_index(drop=True), y_all[tr_idx]
        dtf_tr = dtf_all[tr_idx]
        X_va, y_va = X_all.iloc[va_idx].reset_index(drop=True), y_all[va_idx]

        print(f"\nFold {fold}/{n_splits}: train={len(y_tr):,} (pos={int(y_tr.sum()):,}) | val={len(y_va):,} (pos={int(y_va.sum()):,})")

        Xb, yb = sample_negatives_hard(
            X_tr, y_tr, dtf_tr,
            lookahead=lookahead_days,
            neg_pos_ratio=neg_pos_ratio,
            hard_window=hard_window,
            hard_fraction=hard_fraction,
            seed=random_state,
        )
        print(f"  After hard-neg sampling: {len(yb):,} (pos={int(yb.sum()):,}, neg={len(yb)-int(yb.sum()):,})")

        model = train_rf(
            Xb, yb,
            n_estimators=rf_n_estimators,
            max_depth=rf_max_depth,
            min_samples_leaf=rf_min_samples_leaf,
            max_features=rf_max_features,
            random_state=random_state,
        )

        proba_va = model.predict_proba(X_va)[:, 1]
        thr, pr_auc = pick_threshold(
            y_va, proba_va,
            min_precision=min_precision,
            min_recall=min_recall,
            top_k_rate=top_k_rate,
            min_alerts=min_alerts,
        )
        oof_proba[va_idx] = proba_va

        m = metrics_at_threshold(y_va, proba_va, thr)
        m.update({'pr_auc': float(pr_auc), 'threshold': float(thr), 'fold': int(fold)})
        fold_metrics.append(m)

        del X_tr, y_tr, X_va, y_va, Xb, yb, model
        cleanup()

    # ---------- Global threshold from OOF ----------
    print("\nAggregating OOF predictions to choose a global threshold...")
    thr_global, pr_auc_oof = pick_threshold(
        y_all, oof_proba,
        min_precision=min_precision,
        min_recall=min_recall,
        top_k_rate=top_k_rate,
        min_alerts=min_alerts,
    )
    agg_metrics = metrics_at_threshold(y_all, oof_proba, thr_global)
    agg_metrics.update({'pr_auc': float(pr_auc_oof), 'threshold': float(thr_global)})

    print("\nOOF Performance (using global threshold):")
    print(json.dumps(agg_metrics, indent=2))

    # ---------- FINAL model ----------
    Xb_full, yb_full = sample_negatives_hard(
        X_all, y_all, dtf_all,
        lookahead=lookahead_days,
        neg_pos_ratio=neg_pos_ratio,
        hard_window=hard_window,
        hard_fraction=hard_fraction,
        seed=random_state,
    )
    final_model = train_rf(
        Xb_full, yb_full,
        n_estimators=rf_n_estimators,
        max_depth=rf_max_depth,
        min_samples_leaf=rf_min_samples_leaf,
        max_features=rf_max_features,
        random_state=random_state,
    )
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_prefix = os.path.join(output_dir, f"{dataset_type}_precv5s4_{timestamp}")
    joblib.dump(final_model, f"{model_prefix}_model.pkl")
    with open(f"{model_prefix}_features.json", 'w') as f:
        json.dump({'feature_names': feature_names, 'threshold': float(thr_global), 'category_maps': cat_maps}, f, indent=2)
    print(f"\n✓ Final model saved: {model_prefix}_model.pkl")

    # ---------- TEST (RAM normal) ----------
    test_metrics = None
    if not df_test.empty and (not test_big_mode):
        proba_test = final_model.predict_proba(X_test)[:, 1]
        dtf_test = compute_days_to_failure(df_test)
        y_test = create_labels_from_dtf(dtf_test, lookahead_days)
        test_metrics = metrics_at_threshold(y_test, proba_test, thr_global)
        test_metrics.update({'pr_auc': float(average_precision_score(y_test, proba_test)), 'threshold_used': float(thr_global)})

    # ---------- Metadata ----------
    metadata = {
        'dataset_type': dataset_type,
        'train_years': train_years,
        'test_years': test_years,
        'lookahead_days': lookahead_days,
        'n_splits': n_splits,
        'neg_pos_ratio': neg_pos_ratio,
        'hard_window': hard_window,
        'hard_fraction': hard_fraction,
        'rf_params': {
            'n_estimators': rf_n_estimators,
            'max_depth': rf_max_depth,
            'min_samples_leaf': rf_min_samples_leaf,
            'max_features': rf_max_features,
        },
        'min_precision': min_precision,
        'min_recall': min_recall,
        'top_k_rate': top_k_rate,
        'min_alerts': min_alerts,
        'oof_metrics': agg_metrics,
        'fold_metrics': fold_metrics,
        'test_metrics': test_metrics,
        'feature_names': feature_names,
        'smart_cols': smart_cols
    }

    meta_path = os.path.join(output_dir, f"{dataset_type}_precv5s4_{timestamp}_metadata.json")
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"\n✓ Metadata saved: {meta_path}")
    print("="*100)
    return metadata


# ===================== TEST-ONLY 2024 (STREAMING, BIG) =====================

def evaluate_saved_model_2024_streaming(
    model_path: str,
    features_meta_path: str,  # el *_features.json (con feature_names, thr, cat_maps)
    parquet_path: str,
    train_years: List[int],   # para continuidad categórica, no cargamos train completo
    test_years: List[int],    # usualmente [2024]
    lookahead_days: int = 7,
    chunk_limit_rows: int = 0 # 0 = todos los row-groups
):
    """
    Evalúa 2024 en streaming:
      • NO carga todo 2024 a RAM.
      • Construye labels con fail_map (primera fecha de fallo).
      • Features sin rolling (delta, cummax, age_days) con estado por serial.
      • Devuelve métricas a nivel row y disk usando thr del meta.
    """
    print("="*100)
    print("EVALUATE SAVED MODEL — TEST 2024 (STREAMING)")
    print("="*100)

    model = joblib.load(model_path)
    with open(features_meta_path, "r") as f:
        meta = json.load(f)
    feature_names = meta["feature_names"]
    thr = float(meta.get("threshold", 0.5))
    cat_maps = meta.get("category_maps", {})

    smart_cols = discover_smart_columns(parquet_path)
    base_cols = ['serial_number', 'date', 'failure', 'model']
    read_cols = base_cols + smart_cols

    # Labeling por fail_map del TEST
    fail_map_test = scan_fail_dates(parquet_path, test_years)

    builder = StreamDeltaCummaxBuilder(smart_cols=smart_cols, cat_maps=cat_maps)

    # Acumuladores de métricas en streaming (para umbral thr)
    tn=fp=fn=tp=0
    # Para métricas por disco: guardamos max proba y max label por serial al vuelo
    disk_stat: Dict[str, Tuple[float, int]] = {}  # serial -> (max_proba, max_y)

    processed_groups = 0
    for df in _iter_parquet_row_groups(parquet_path, test_years, columns=read_cols):
        if chunk_limit_rows and processed_groups >= chunk_limit_rows:
            break

        # Labels con lookahead
        sn_ser = df["serial_number"].astype(str)
        dt = pd.to_datetime(df["date"], errors="coerce").to_numpy(dtype="datetime64[D]")
        fdt = pd.to_datetime(sn_ser.map(fail_map_test), errors="coerce").to_numpy(dtype="datetime64[D]")
        dtf = np.full(len(df), 10**9, dtype=np.int64)
        valid = ~np.isnat(fdt)
        if valid.any():
            dd = (fdt[valid] - dt[valid]).astype("timedelta64[D]").astype("int64")
            dtf[valid] = dd
        y_chunk = ((dtf >= 0) & (dtf <= lookahead_days)).astype(np.int8)

        # Features streaming sin rolling
        X_chunk = builder.transform_chunk(df)

        # Reordena columnas faltantes/sobrantes para el modelo
        for m in feature_names:
            if m not in X_chunk.columns:
                X_chunk[m] = 0.0
        X_chunk = X_chunk[feature_names].astype(np.float32)

        proba = model.predict_proba(X_chunk)[:, 1]
        y_pred = (proba >= thr).astype(np.int8)

        # Confusion row-level
        cm = confusion_matrix(y_chunk, y_pred, labels=[0,1])
        tn += int(cm[0,0]); fp += int(cm[0,1]); fn += int(cm[1,0]); tp += int(cm[1,1])

        # Disk-level: actualiza max proba y max y por serial
        for s, p, y in zip(sn_ser.values, proba, y_chunk):
            if s not in disk_stat:
                disk_stat[s] = (p, int(y))
            else:
                mp, my = disk_stat[s]
                disk_stat[s] = (max(mp, p), max(my, int(y)))

        processed_groups += 1
        cleanup()

    # Métricas fila
    row_metrics = {
        'precision': float(tp / max(1, tp + fp)),
        'recall': float(tp / max(1, tp + fn)),
        'f1': float((2*tp) / max(1, 2*tp + fp + fn)),
        'confusion_matrix': [[tn, fp], [fn, tp]],
        'fpr': float(fp / max(1, fp + tn))
    }

    # Métricas por disco
    y_disk = np.array([v[1] for v in disk_stat.values()], dtype=np.int8)
    yhat_disk = np.array([1 if v[0] >= thr else 0 for v in disk_stat.values()], dtype=np.int8)
    cm_d = confusion_matrix(y_disk, yhat_disk, labels=[0,1])
    tn_d, fp_d, fn_d, tp_d = int(cm_d[0,0]), int(cm_d[0,1]), int(cm_d[1,0]), int(cm_d[1,1])
    disk_metrics = {
        'precision': float(tp_d / max(1, tp_d + fp_d)),
        'recall': float(tp_d / max(1, tp_d + fn_d)),
        'f1': float((2*tp_d) / max(1, 2*tp_d + fp_d + fn_d)),
        'confusion_matrix': [[tn_d, fp_d], [fn_d, tp_d]],
        'n_disks': int(len(disk_stat))
    }

    print("\nRow-level TEST metrics (streaming 2024):")
    print(json.dumps(row_metrics, indent=2))
    print("\nDisk-level TEST metrics (streaming 2024):")
    print(json.dumps(disk_metrics, indent=2))
    print("="*100)
    return {"row": row_metrics, "disk": disk_metrics}


# ===================== WRAPPERS =====================

def train_ssd_precision_v5s4():
    # SSD cabe en RAM → rolling ON, test normal
    return train_random_forest_precision_pipeline(
        train_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        dataset_type='SSD',
        train_years=[2020, 2021, 2022, 2023],
        test_years=[2024],
        lookahead_days=7,
        n_splits=5,
        neg_pos_ratio=3,
        hard_window=60,
        hard_fraction=0.7,
        rf_n_estimators=500,
        rf_max_depth=16,
        rf_min_samples_leaf=20,
        rf_max_features=0.3,
        min_precision=0.85,
        min_recall=0.05,
        top_k_rate=2e-4,
        min_alerts=20,
        output_dir='./models_rf_ssd_precv5s4',
        big_mode=False,
        test_big_mode=False,
        neg_random_keep_rate=0.0025,
        add_rolling=True
    )

def train_hdd_precision_v5s4(neg_random_keep_rate: float = 0.0025):
    # HDD enorme → TRAIN big + TEST streaming; rolling OFF para poder hacer streaming
    return train_random_forest_precision_pipeline(
        train_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        dataset_type='HDD',
        train_years=[2020, 2021, 2022, 2023],
        test_years=[2024],
        lookahead_days=7,
        n_splits=5,
        neg_pos_ratio=3,
        hard_window=90,
        hard_fraction=0.7,
        rf_n_estimators=400,
        rf_max_depth=16,
        rf_min_samples_leaf=20,
        rf_max_features=0.3,
        min_precision=0.85,
        min_recall=0.05,
        top_k_rate=1e-4,
        min_alerts=20,
        output_dir='./models_rf_hdd_precv5s4',
        big_mode=True,
        test_big_mode=True,
        neg_random_keep_rate=neg_random_keep_rate,
        add_rolling=False
    )


In [4]:
train_ssd_precision_v5s4()

RANDOM FOREST (PRECISION-CONTROLLED v5s4) - SSD
Loading [2020, 2021, 2022, 2023] from ./Procesados/finales/SSD_FULL_CLEAN.parquet...
Loaded 2,124,111 rows
Loading [2024] from ./Procesados/finales/SSD_FULL_CLEAN.parquet...
Loaded 1,220,745 rows
Labels (train): 1,325 positive (0.0624%)
Creating features (temporal join) for SSD...
  Found 13 SMART attributes
  Final features: 71

Fold 1/5: train=1,709,653 (pos=1,079) | val=414,458 (pos=246)
  After hard-neg sampling: 4,316 (pos=1,079, neg=3,237)
  Train RF: n=4,316 (pos=1,079) | trees=500 depth=16 leaf=20

Fold 2/5: train=1,707,701 (pos=1,054) | val=416,410 (pos=271)
  After hard-neg sampling: 4,216 (pos=1,054, neg=3,162)
  Train RF: n=4,216 (pos=1,054) | trees=500 depth=16 leaf=20

Fold 3/5: train=1,682,081 (pos=1,061) | val=442,030 (pos=264)
  After hard-neg sampling: 4,244 (pos=1,061, neg=3,183)
  Train RF: n=4,244 (pos=1,061) | trees=500 depth=16 leaf=20

Fold 4/5: train=1,691,757 (pos=1,061) | val=432,354 (pos=264)
  After hard-neg s

{'dataset_type': 'SSD',
 'train_years': [2020, 2021, 2022, 2023],
 'test_years': [2024],
 'lookahead_days': 7,
 'n_splits': 5,
 'neg_pos_ratio': 3,
 'hard_window': 60,
 'hard_fraction': 0.7,
 'rf_params': {'n_estimators': 500,
  'max_depth': 16,
  'min_samples_leaf': 20,
  'max_features': 0.3},
 'min_precision': 0.85,
 'min_recall': 0.05,
 'top_k_rate': 0.0002,
 'min_alerts': 20,
 'oof_metrics': {'precision': 0.8514056224899599,
  'recall': 0.16,
  'f1': 0.26937738246505716,
  'confusion_matrix': [[2122749, 37], [1113, 212]],
  'fpr': 1.7429924636774504e-05,
  'pr_auc': 0.25883732136200555,
  'threshold': 0.8173822164535522},
 'fold_metrics': [{'precision': 0.8507462686567164,
   'recall': 0.23170731707317074,
   'f1': 0.36421725239616615,
   'confusion_matrix': [[414202, 10], [189, 57]],
   'fpr': 2.414222668585169e-05,
   'pr_auc': 0.28124776008354263,
   'threshold': 0.6681490961467218,
   'fold': 1},
  {'precision': 0.8548387096774194,
   'recall': 0.19557195571955718,
   'f1': 0.3

In [3]:
train_hdd_precision_v5s4()

RANDOM FOREST (PRECISION-CONTROLLED v5s4) - HDD
Scanning earliest failure dates (streaming)...
  Found 8,044 failed serials
Loading BIG-FILTERED [2020, 2021, 2022, 2023] from ./Procesados/finales/HDD_FULL_CLEAN.parquet (keep_rate=0.0025)...
  BIG-FILTERED kept 1,184,183/215,762,596 rows (~0.55%)
Labels (train): 62,131 positive (5.2467%)
Creating features (temporal join) for HDD...
  Found 5 SMART attributes
  Final features: 20

Fold 1/5: train=948,389 (pos=49,699) | val=235,794 (pos=12,432)
  After hard-neg sampling: 198,796 (pos=49,699, neg=149,097)
  Train RF: n=198,796 (pos=49,699) | trees=400 depth=16 leaf=20

Fold 2/5: train=946,580 (pos=49,693) | val=237,603 (pos=12,438)
  After hard-neg sampling: 198,772 (pos=49,693, neg=149,079)
  Train RF: n=198,772 (pos=49,693) | trees=400 depth=16 leaf=20

Fold 3/5: train=948,183 (pos=49,752) | val=236,000 (pos=12,379)
  After hard-neg sampling: 199,008 (pos=49,752, neg=149,256)
  Train RF: n=199,008 (pos=49,752) | trees=400 depth=16 leaf=2

{'dataset_type': 'HDD',
 'train_years': [2020, 2021, 2022, 2023],
 'test_years': [2024],
 'lookahead_days': 7,
 'n_splits': 5,
 'neg_pos_ratio': 3,
 'hard_window': 90,
 'hard_fraction': 0.7,
 'rf_params': {'n_estimators': 400,
  'max_depth': 16,
  'min_samples_leaf': 20,
  'max_features': 0.3},
 'min_precision': 0.85,
 'min_recall': 0.05,
 'top_k_rate': 0.0001,
 'min_alerts': 20,
 'oof_metrics': {'precision': 0.85000591326091,
  'recall': 0.8097568041718305,
  'f1': 0.8293933399274646,
  'confusion_matrix': [[1113174, 8878], [11820, 50311]],
  'fpr': 0.007912289270016007,
  'pr_auc': 0.8902504742465743,
  'threshold': 0.5678298473358154},
 'fold_metrics': [{'precision': 0.8500422654268808,
   'recall': 0.8088803088803089,
   'f1': 0.8289506223724342,
   'confusion_matrix': [[221588, 1774], [2376, 10056]],
   'fpr': 0.007942264127291123,
   'pr_auc': 0.8899268014273016,
   'threshold': 0.5486192780535112,
   'fold': 1},
  {'precision': 0.8500588334173811,
   'recall': 0.813153240070751,

# Demás Técnicas

In [2]:
"""
CPU-ONLY RANDOM FOREST PIPELINE
Works without GPU - uses scikit-learn instead of cuML

Same functionality, just slower but will actually run!
"""

import os, gc, json, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, average_precision_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
import joblib

warnings.filterwarnings('ignore')

def cleanup():
    gc.collect()

# ============== LOAD DATA ==============
def load_data(path, years):
    print(f"Loading {years}...")
    chunks = []
    if os.path.isdir(path):
        files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parquet')]
    else:
        files = [path]
    
    for f in files:
        pf = pq.ParquetFile(f)
        for i in range(pf.num_row_groups):
            df = pf.read_row_group(i).to_pandas()
            if 'date' not in df.columns: continue
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df = df[df['date'].dt.year.isin(years)]
            if len(df) > 0: chunks.append(df)
            if len(chunks) >= 20:
                chunks = [pd.concat(chunks, ignore_index=True)]
                cleanup()
    
    result = pd.concat(chunks, ignore_index=True) if chunks else pd.DataFrame()
    print(f"Loaded {len(result):,} rows")
    return result

# ============== CREATE LABELS ==============
def create_labels(df, lookahead=7):
    print(f"Creating labels (lookahead={lookahead})...")
    df['date'] = pd.to_datetime(df['date'])
    
    fail_map = {}
    fails = df[df['failure'] == 1]
    for sn, dt in zip(fails['serial_number'], fails['date']):
        sn = str(sn)
        if sn not in fail_map:
            fail_map[sn] = dt
        else:
            fail_map[sn] = min(fail_map[sn], dt)
    
    y = np.zeros(len(df), dtype=np.int8)
    for i, (sn, dt) in enumerate(zip(df['serial_number'].astype(str), df['date'])):
        if sn in fail_map:
            days = (fail_map[sn] - dt).days
            if 0 <= days <= lookahead:
                y[i] = 1
    
    print(f"Labels: {y.sum():,} positive ({100*y.sum()/len(y):.4f}%)")
    return y

# ============== FEATURES ==============
def create_features(df, dataset_type):
    """Uses ALL available SMART attributes from the data."""
    print(f"Creating features for {dataset_type}...")
    df = df.sort_values(['serial_number', 'date']).copy()
    
    # Auto-detect SMART attributes
    all_cols = df.columns.tolist()
    smart_cols = [c for c in all_cols if 'smart' in c.lower() and '_raw' in c.lower()]
    
    print(f"  Found {len(smart_cols)} SMART attributes")
    
    # Ensure numeric
    for c in smart_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)
    
    # Delta features
    for c in smart_cols:
        df[f'delta_{c}'] = df.groupby('serial_number')[c].diff().fillna(0)
    
    # Cumulative max
    for c in smart_cols:
        df[f'max_{c}'] = df.groupby('serial_number')[c].cummax()
    
    # Age
    df['age_days'] = df.groupby('serial_number').cumcount()
    
    # Temporal
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek
    
    # Prepare
    drop_cols = ['serial_number', 'date', 'failure', 'model']
    X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
    
    for c in X.columns:
        if X[c].dtype == 'object':
            X[c] = pd.Categorical(X[c]).codes
        X[c] = pd.to_numeric(X[c], errors='coerce').fillna(0)
    
    # Remove zero-variance
    var = X.var()
    keep_cols = var[var > 0].index.tolist()
    X = X[keep_cols]
    
    print(f"  Final features: {len(X.columns)}")
    
    return X.astype(np.float32)

# ============== BALANCING ==============
def apply_balance(X, y, strategy, target=50000, seed=42):
    n_pos = int(y.sum())
    n_neg = len(y) - n_pos
    print(f"\n{strategy.upper()} balancing:")
    print(f"  Before: {len(y):,} samples ({n_pos:,} pos, {n_neg:,} neg)")
    
    if strategy == 'none':
        print(f"  After: {len(y):,} samples (no resampling)")
        return X, y
    
    elif strategy == 'under':
        n_per_class = min(target // 2, n_pos, n_neg)
        rus = RandomUnderSampler(
            sampling_strategy={0: n_per_class, 1: n_per_class},
            random_state=seed
        )
        X_res, y_res = rus.fit_resample(X, y)
        print(f"  After: {len(y_res):,} samples ({y_res.sum():,} pos)")
        return X_res, y_res
    
    elif strategy == 'smote':
        if n_pos < 6:
            print("  Not enough positives for SMOTE, using under")
            return apply_balance(X, y, 'under', target, seed)
        
        max_neg = min(n_pos * 20, n_neg)
        if max_neg < n_neg:
            rus = RandomUnderSampler(
                sampling_strategy={0: max_neg, 1: n_pos},
                random_state=seed
            )
            X, y = rus.fit_resample(X, y)
            n_neg = max_neg
        
        target_pos = min(n_neg // 2, target // 2, n_pos * 3)
        k = min(5, n_pos - 1)
        
        sm = SMOTE(
            sampling_strategy={1: target_pos},
            k_neighbors=k,
            random_state=seed
        )
        X_res, y_res = sm.fit_resample(X, y)
        
        if len(X_res) > target:
            idx = np.random.choice(len(X_res), target, replace=False)
            X_res, y_res = X_res.iloc[idx], y_res[idx]
        
        print(f"  After: {len(y_res):,} samples ({y_res.sum():,} pos)")
        return X_res, y_res
    
    elif strategy == 'smote_enn':
        if n_pos < 6:
            print("  Not enough positives for SMOTE-ENN, using under")
            return apply_balance(X, y, 'under', target, seed)
        
        max_neg = min(n_pos * 20, n_neg)
        if max_neg < n_neg:
            rus = RandomUnderSampler(
                sampling_strategy={0: max_neg, 1: n_pos},
                random_state=seed
            )
            X, y = rus.fit_resample(X, y)
        
        print("  Applying SMOTE-ENN (slow)...")
        smenn = SMOTEENN(random_state=seed)
        X_res, y_res = smenn.fit_resample(X, y)
        
        if len(X_res) > target:
            idx = np.random.choice(len(X_res), target, replace=False)
            X_res, y_res = X_res.iloc[idx], y_res[idx]
        
        print(f"  After: {len(y_res):,} samples ({y_res.sum():,} pos)")
        return X_res, y_res
    
    raise ValueError(f"Unknown strategy: {strategy}")

# ============== TRAIN ==============
def train_rf(X, y, use_class_weights=True, n_estimators=300, max_depth=20, random_state=42):
    """Train Random Forest on CPU using scikit-learn"""
    print(f"\nTraining Random Forest (CPU):")
    print(f"  Samples: {len(y):,} ({y.sum():,} pos)")
    print(f"  Trees: {n_estimators}, Depth: {max_depth}")
    
    # Calculate class weights
    class_weight = None
    if use_class_weights and y.sum() < len(y) // 2:
        n_samples = len(y)
        n_pos = y.sum()
        n_neg = n_samples - n_pos
        
        w_pos = n_samples / (2 * n_pos) if n_pos > 0 else 1.0
        w_neg = n_samples / (2 * n_neg) if n_neg > 0 else 1.0
        
        class_weight = {0: w_neg, 1: w_pos}
        print(f"  Class weights: pos={w_pos:.1f}, neg={w_neg:.1f}")
    
    # Train
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=0.5,
        class_weight=class_weight,
        random_state=random_state,
        n_jobs=-1,  # Use all CPU cores
        verbose=0
    )
    
    rf.fit(X, y)
    print("  Training complete!")
    
    return rf

# ============== EVALUATE ==============
def evaluate(model, X, y, feature_names, min_precision=0.60, min_recall=0.30):
    print(f"\nEvaluating:")
    print(f"  Samples: {len(y):,} ({y.sum():,} pos)")
    
    # Ensure same features
    X = X[feature_names]
    
    # Predict
    proba = model.predict_proba(X)[:, 1]
    
    # PR curve
    precision, recall, thresholds = precision_recall_curve(y, proba)
    pr_auc = average_precision_score(y, proba)
    
    print(f"  PR-AUC: {pr_auc:.6f}")
    
    # Find threshold
    beta = 2.0
    f_beta = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall + 1e-10)
    
    valid_mask = (precision >= min_precision) & (recall >= min_recall)
    
    if valid_mask.any():
        idx = np.argmax(f_beta * valid_mask)
        threshold = thresholds[idx]
        print(f"  Optimal threshold: {threshold:.4f}")
        print(f"    Precision: {precision[idx]:.4f}")
        print(f"    Recall: {recall[idx]:.4f}")
    else:
        idx = np.argmax(f_beta)
        threshold = thresholds[idx]
        print(f"  ⚠️  No threshold meets constraints")
        print(f"  Best F2 threshold: {threshold:.4f}")
        print(f"    Precision: {precision[idx]:.4f}")
        print(f"    Recall: {recall[idx]:.4f}")
    
    # Metrics
    y_pred = (proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    
    print(f"\n  Confusion Matrix:")
    print(f"    TN: {tn:,}  |  FP: {fp:,}")
    print(f"    FN: {fn:,}  |  TP: {tp:,}")
    print(f"  FPR: {fp/(fp+tn):.6f}")
    
    return {
        'pr_auc': float(pr_auc),
        'threshold': float(threshold),
        'precision': float(precision_score(y, y_pred, zero_division=0)),
        'recall': float(recall_score(y, y_pred, zero_division=0)),
        'f1': float(f1_score(y, y_pred, zero_division=0)),
        'confusion_matrix': [[int(tn), int(fp)], [int(fn), int(tp)]],
        'fpr': float(fp / (fp + tn)) if (fp + tn) > 0 else 0.0
    }

# ============== MAIN PIPELINE ==============
def train_random_forest_pipeline(
    train_parquet,
    test_parquet,
    dataset_type,
    balancing_strategy='none',
    train_years=[2020, 2021, 2022, 2023],
    test_years=[2024],
    lookahead_days=7,
    n_estimators=300,
    max_depth=20,
    target_samples=50000,
    min_precision=0.60,
    min_recall=0.30,
    output_dir='./models_rf',
    random_state=42
):
    print("="*70)
    print(f"RANDOM FOREST PIPELINE (CPU) - {dataset_type.upper()}")
    print(f"Strategy: {balancing_strategy.upper()}")
    print("="*70)
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Load
    df_train = load_data(train_parquet, train_years)
    df_test = load_data(test_parquet, test_years) if test_parquet else pd.DataFrame()
    
    if df_train.empty:
        raise ValueError("Training data is empty!")
    
    # Labels
    y_train = create_labels(df_train, lookahead_days)
    y_test = create_labels(df_test, lookahead_days) if not df_test.empty else np.array([])
    
    if y_train.sum() < 50:
        raise ValueError(f"Insufficient positive samples: {y_train.sum()}")
    
    # Features
    X_train = create_features(df_train, dataset_type)
    X_test = create_features(df_test, dataset_type) if not df_test.empty else pd.DataFrame()
    
    feature_names = list(X_train.columns)
    print(f"\nTotal features: {len(feature_names)}")
    
    if not X_test.empty:
        for col in feature_names:
            if col not in X_test.columns:
                X_test[col] = 0
        X_test = X_test[feature_names]
    
    del df_train, df_test
    cleanup()
    
    # Split
    n_val = int(0.2 * len(X_train))
    X_val = X_train.iloc[-n_val:].reset_index(drop=True)
    y_val = y_train[-n_val:]
    
    X_train = X_train.iloc[:-n_val].reset_index(drop=True)
    y_train = y_train[:-n_val]
    
    print(f"\nData splits:")
    print(f"  Training:   {len(y_train):,} ({y_train.sum():,} pos)")
    print(f"  Validation: {len(y_val):,} ({y_val.sum():,} pos)")
    if len(y_test) > 0:
        print(f"  Test:       {len(y_test):,} ({y_test.sum():,} pos)")
    
    # Balance
    X_train_bal, y_train_bal = apply_balance(
        X_train, y_train,
        strategy=balancing_strategy,
        target=target_samples,
        seed=random_state
    )
    
    # Train
    use_weights = (balancing_strategy == 'none')
    model = train_rf(
        X_train_bal, y_train_bal,
        use_class_weights=use_weights,
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state
    )
    
    del X_train, y_train, X_train_bal, y_train_bal
    cleanup()
    
    # Evaluate
    val_metrics = evaluate(model, X_val, y_val, feature_names, min_precision, min_recall)
    
    del X_val, y_val
    cleanup()
    
    test_metrics = None
    if not X_test.empty:
        test_metrics = evaluate(model, X_test, y_test, feature_names, min_precision, min_recall)
        del X_test, y_test
        cleanup()
    
    # Save
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_prefix = os.path.join(output_dir, f"{dataset_type}_{balancing_strategy}_{timestamp}")
    
    joblib.dump(model, f"{model_prefix}_model.pkl")
    print(f"\n✓ Model saved: {model_prefix}_model.pkl")
    
    metadata = {
        'dataset_type': dataset_type,
        'balancing_strategy': balancing_strategy,
        'train_years': train_years,
        'test_years': test_years,
        'lookahead_days': lookahead_days,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'feature_names': feature_names,
        'validation_metrics': val_metrics,
        'test_metrics': test_metrics
    }
    
    with open(f"{model_prefix}_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✓ Metadata saved")
    print("="*70)
    
    return metadata


# ============== INDIVIDUAL FUNCTIONS ==============

# HDD
def train_hdd_none():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        dataset_type='HDD',
        balancing_strategy='none',
        lookahead_days=7,
        n_estimators=300,
        max_depth=20,
        target_samples=50000,
        output_dir='./models_rf_hdd_none'
    )

def train_hdd_under():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        dataset_type='HDD',
        balancing_strategy='under',
        lookahead_days=7,
        n_estimators=300,
        max_depth=20,
        target_samples=50000,
        output_dir='./models_rf_hdd_under'
    )

def train_hdd_smote():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        dataset_type='HDD',
        balancing_strategy='smote',
        lookahead_days=7,
        n_estimators=300,
        max_depth=20,
        target_samples=50000,
        output_dir='./models_rf_hdd_smote'
    )

def train_hdd_smote_enn():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/HDD_FULL_CLEAN.parquet',
        dataset_type='HDD',
        balancing_strategy='smote_enn',
        lookahead_days=7,
        n_estimators=300,
        max_depth=20,
        target_samples=50000,
        output_dir='./models_rf_hdd_smote_enn'
    )

# SSD
def train_ssd_none():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        dataset_type='SSD',
        balancing_strategy='none',
        lookahead_days=7,
        n_estimators=400,
        max_depth=25,
        target_samples=30000,
        min_precision=0.50,
        min_recall=0.25,
        output_dir='./models_rf_ssd_none'
    )

def train_ssd_under():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        dataset_type='SSD',
        balancing_strategy='under',
        lookahead_days=7,
        n_estimators=400,
        max_depth=25,
        target_samples=30000,
        min_precision=0.50,
        min_recall=0.25,
        output_dir='./models_rf_ssd_under'
    )

def train_ssd_smote():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        dataset_type='SSD',
        balancing_strategy='smote',
        lookahead_days=7,
        n_estimators=400,
        max_depth=25,
        target_samples=30000,
        min_precision=0.50,
        min_recall=0.25,
        output_dir='./models_rf_ssd_smote'
    )

def train_ssd_smote_enn():
    return train_random_forest_pipeline(
        train_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        test_parquet='./Procesados/finales/SSD_FULL_CLEAN.parquet',
        dataset_type='SSD',
        balancing_strategy='smote_enn',
        lookahead_days=7,
        n_estimators=400,
        max_depth=25,
        target_samples=30000,
        min_precision=0.50,
        min_recall=0.25,
        output_dir='./models_rf_ssd_smote_enn'
    )



In [4]:
train_ssd_none()

RANDOM FOREST PIPELINE (CPU) - SSD
Strategy: NONE
Loading [2020, 2021, 2022, 2023]...
Loaded 2,124,111 rows
Loading [2024]...
Loaded 1,220,745 rows
Creating labels (lookahead=7)...
Labels: 1,325 positive (0.0624%)
Creating labels (lookahead=7)...
Labels: 227 positive (0.0186%)
Creating features for SSD...
  Found 13 SMART attributes
  Final features: 40
Creating features for SSD...
  Found 13 SMART attributes
  Final features: 42

Total features: 40

Data splits:
  Training:   1,699,289 (1,240 pos)
  Validation: 424,822 (85 pos)
  Test:       1,220,745 (227 pos)

NONE balancing:
  Before: 1,699,289 samples (1,240 pos, 1,698,049 neg)
  After: 1,699,289 samples (no resampling)

Training Random Forest (CPU):
  Samples: 1,699,289 (1,240 pos)
  Trees: 400, Depth: 25
  Class weights: pos=685.2, neg=0.5
  Training complete!

Evaluating:
  Samples: 424,822 (85 pos)
  PR-AUC: 0.000251
  ⚠️  No threshold meets constraints
  Best F2 threshold: 0.2239
    Precision: 0.0005
    Recall: 0.0235

  Co

{'dataset_type': 'SSD',
 'balancing_strategy': 'none',
 'train_years': [2020, 2021, 2022, 2023],
 'test_years': [2024],
 'lookahead_days': 7,
 'n_estimators': 400,
 'max_depth': 25,
 'feature_names': ['capacity_bytes',
  'smart_5_raw',
  'smart_187_raw',
  'smart_194_raw',
  'smart_231_raw',
  'smart_233_raw',
  'smart_241_raw',
  'smart_242_raw',
  'smart_9_raw',
  'smart_173_raw',
  'smart_174_raw',
  'smart_199_raw',
  'smart_230_raw',
  'delta_smart_5_raw',
  'delta_smart_187_raw',
  'delta_smart_194_raw',
  'delta_smart_231_raw',
  'delta_smart_233_raw',
  'delta_smart_241_raw',
  'delta_smart_242_raw',
  'delta_smart_9_raw',
  'delta_smart_173_raw',
  'delta_smart_174_raw',
  'delta_smart_199_raw',
  'delta_smart_230_raw',
  'max_smart_5_raw',
  'max_smart_187_raw',
  'max_smart_194_raw',
  'max_smart_231_raw',
  'max_smart_233_raw',
  'max_smart_241_raw',
  'max_smart_242_raw',
  'max_smart_9_raw',
  'max_smart_173_raw',
  'max_smart_174_raw',
  'max_smart_199_raw',
  'max_smart

In [3]:
train_ssd_smote()

RANDOM FOREST PIPELINE (CPU) - SSD
Strategy: SMOTE
Loading [2020, 2021, 2022, 2023]...
Loaded 2,124,111 rows
Loading [2024]...
Loaded 1,220,745 rows
Creating labels (lookahead=7)...
Labels: 1,325 positive (0.0624%)
Creating labels (lookahead=7)...
Labels: 227 positive (0.0186%)
Creating features for SSD...
  Found 13 SMART attributes
  Final features: 40
Creating features for SSD...
  Found 13 SMART attributes
  Final features: 42

Total features: 40

Data splits:
  Training:   1,699,289 (1,240 pos)
  Validation: 424,822 (85 pos)
  Test:       1,220,745 (227 pos)

SMOTE balancing:
  Before: 1,699,289 samples (1,240 pos, 1,698,049 neg)
  After: 28,520 samples (3,720 pos)

Training Random Forest (CPU):
  Samples: 28,520 (3,720 pos)
  Trees: 400, Depth: 25
  Training complete!

Evaluating:
  Samples: 424,822 (85 pos)
  PR-AUC: 0.000239
  ⚠️  No threshold meets constraints
  Best F2 threshold: 0.3586
    Precision: 0.0005
    Recall: 0.0118

  Confusion Matrix:
    TN: 422,864  |  FP: 1,87

{'dataset_type': 'SSD',
 'balancing_strategy': 'smote',
 'train_years': [2020, 2021, 2022, 2023],
 'test_years': [2024],
 'lookahead_days': 7,
 'n_estimators': 400,
 'max_depth': 25,
 'feature_names': ['capacity_bytes',
  'smart_5_raw',
  'smart_187_raw',
  'smart_194_raw',
  'smart_231_raw',
  'smart_233_raw',
  'smart_241_raw',
  'smart_242_raw',
  'smart_9_raw',
  'smart_173_raw',
  'smart_174_raw',
  'smart_199_raw',
  'smart_230_raw',
  'delta_smart_5_raw',
  'delta_smart_187_raw',
  'delta_smart_194_raw',
  'delta_smart_231_raw',
  'delta_smart_233_raw',
  'delta_smart_241_raw',
  'delta_smart_242_raw',
  'delta_smart_9_raw',
  'delta_smart_173_raw',
  'delta_smart_174_raw',
  'delta_smart_199_raw',
  'delta_smart_230_raw',
  'max_smart_5_raw',
  'max_smart_187_raw',
  'max_smart_194_raw',
  'max_smart_231_raw',
  'max_smart_233_raw',
  'max_smart_241_raw',
  'max_smart_242_raw',
  'max_smart_9_raw',
  'max_smart_173_raw',
  'max_smart_174_raw',
  'max_smart_199_raw',
  'max_smar

In [8]:
train_ssd_under()


=== SSD | Balance=under | Lookahead=2d ===
RF params: {'n_estimators': 250, 'max_depth': 18, 'max_features': 0.35, 'n_bins': 128, 'random_state': 42, 'n_streams': 1}

--- Fold 1 | Train [2020, 2021, 2022] | Val [2023] ---
Train: 792 rows (396 pos, 50.00%)
Fold 1: PR-AUC≈0.000163 (rows=1,137,101, pos=104)

--- Fold 2 | Train [2020, 2023] | Val [2021, 2022] ---
Train: 138 rows (69 pos, 50.00%)
Fold 2: PR-AUC≈0.000369 (rows=705,084, pos=395)

--- Fold 3 | Train [2021, 2022, 2023] | Val [2020] ---
Train: 864 rows (432 pos, 50.00%)
Fold 3: PR-AUC≈0.000048 (rows=281,926, pos=10)

Selected threshold: 0.2320 (precision_target=0.95)

Held-out 2024 Test (streamed + persistence):
  Precision=0.000074 | Recall=0.781609 | F1=0.000148 | PR-AUC≈0.000123
  Persistence: k=2, n=3 | Alerts=921,274
  Confusion Matrix: [[TN=299452, FP=921206], [FN=19, TP=68]]


{'cv_metrics': [{'fold': 1,
   'val_rows': 1137101,
   'val_pos': 104,
   'pr_auc_approx': 0.0001630692760735205},
  {'fold': 2,
   'val_rows': 705084,
   'val_pos': 395,
   'pr_auc_approx': 0.0003685138678518559},
  {'fold': 3,
   'val_rows': 281926,
   'val_pos': 10,
   'pr_auc_approx': 4.827346895895134e-05}],
 'threshold': 0.231990231990232,
 'saved_prefix': './models_rf_temporal_ssd/SSD_RFgpu_temporal_under_L2_20251031_165658',
 'test_metrics': {'precision': 7.381083152243523e-05,
  'recall': 0.7816091954022989,
  'f1': 0.0001476077237910287,
  'pr_auc_approx': 0.00012286151845665422,
  'alerts': 921274,
  'positives_rows': 87,
  'confusion_matrix': [[299452, 921206], [19, 68]],
  'persistence_k': 2,
  'persistence_n': 3}}

In [13]:
train_ssd_smote_enn()


=== SSD | Balance=smote_enn | Lookahead=2d ===
RF params: {'n_estimators': 250, 'max_depth': 18, 'max_features': 0.35, 'n_bins': 128, 'random_state': 42, 'n_streams': 1}

--- Fold 1 | Train [2020, 2021, 2022] | Val [2023] ---
Train: 10,000 rows (5,000 pos, 50.00%)
Fold 1: PR-AUC≈0.000135 (rows=1,137,101, pos=104)

--- Fold 2 | Train [2020, 2023] | Val [2021, 2022] ---
Train: 2,510 rows (1,255 pos, 50.00%)
Fold 2: PR-AUC≈0.000647 (rows=705,084, pos=395)

--- Fold 3 | Train [2021, 2022, 2023] | Val [2020] ---
Train: 10,000 rows (5,000 pos, 50.00%)
Fold 3: PR-AUC≈0.000046 (rows=281,926, pos=10)

Selected threshold: 0.2598 (precision_target=0.95)

Held-out 2024 Test (streamed + persistence):
  Precision=0.000076 | Recall=0.701149 | F1=0.000151 | PR-AUC≈0.000106
  Persistence: k=2, n=3 | Alerts=805,366
  Confusion Matrix: [[TN=415353, FP=805305], [FN=26, TP=61]]


{'cv_metrics': [{'fold': 1,
   'val_rows': 1137101,
   'val_pos': 104,
   'pr_auc_approx': 0.00013548275867919293},
  {'fold': 2,
   'val_rows': 705084,
   'val_pos': 395,
   'pr_auc_approx': 0.0006473176056100102},
  {'fold': 3,
   'val_rows': 281926,
   'val_pos': 10,
   'pr_auc_approx': 4.6483844137817164e-05}],
 'threshold': 0.25982905982905985,
 'saved_prefix': './models_rf_temporal_ssd/SSD_RFgpu_temporal_smote_enn_L2_20251031_200543',
 'test_metrics': {'precision': 7.574196079794776e-05,
  'recall': 0.7011494252873564,
  'f1': 0.00015146755924905117,
  'pr_auc_approx': 0.00010550914989970685,
  'alerts': 805366,
  'positives_rows': 87,
  'confusion_matrix': [[415353, 805305], [26, 61]],
  'persistence_k': 2,
  'persistence_n': 3}}

In [ ]:
train_hdd_none()


=== HDD | Balance=none | Lookahead=8d ===
RF params: {'n_estimators': 320, 'max_depth': 18, 'max_features': 0.4, 'n_bins': 128, 'random_state': 42, 'n_streams': 1}

--- Fold 1 | Train [2020, 2021, 2022] | Val [2023] ---
Train: 112,519 rows (32,519 pos, 28.90%)
